In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [16]:
data = pd.read_csv('localhost.csv', sep='delimiter', header=None)

C:\Users\remij\Miniconda3\lib\site-packages\ipykernel_launcher.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  """Entry point for launching an IPython kernel.


In [17]:
data

,0
0,"""1"",""Barchiefs"""
1,"""2"",""Barmen"""
2,"""1"",""1"",""10"""
3,"""2"",""1"",""11"""
4,"""3"",""1"",""25"""
...,...
201430,"""d60f10f1007ebd9f7bb57ba4d0ef3f91ace3e21c8c3c4..."
201431,"""db2bfc7e456fe6eb876aeb9f5041a8401c25762ac97d8..."
201432,"""eb4592a8e95a3016a256e9c55742054733cf2bc687945..."
201433,"""edfc5ba030622c28a77906f8e8374fd569ac23ef9e724..."
